In [1]:
using LinearAlgebra
using Combinatorics
using Statistics
using PyPlot
using BenchmarkTools
using Profile

const X = [0, 1, 1, 0]
const Y = [0, 1im, -1im, 0]
const Z = [1, 0, 0, -1]
const II = [1, 0, 0, 1]

4-element Array{Int64,1}:
 1
 0
 0
 1

In [2]:
function n_choose_k(n, k)
    if k==0
        return [Bool[0 for _ in 1:n]]
    elseif k==n
        return [Bool[1 for _ in 1:n]]
    else
        a = n_choose_k(n-1, k)
        b = n_choose_k(n-1, k-1)
        return [
            [Bool[0, i...] for i in a]..., 
            [Bool[1, i...] for i in b]...
        ]
    end
end

function all_bs(n)
    if n==1
        return [false false]
    else
        tmp = all_bs(n-1)
        tmpp = Array{Bool, 1}[]
        for a in tmp
            push!(tmpp, [0, a...])
            push!(tmpp, [1, a...])
        end
        return tmpp
    end
end

function cal_op(c)
    n = length(c)
    bs = all_bs(n)
    op = zeros(2^n)
    ii = [1, 1]
    zz = [1, -1]
    for a in bs, b in bs
        if sum(a) != sum(b)
            continue
        end
        tmp = [ii + (a[i] ? -1 : 1) * zz for i in 1:n]
        tmpp = kron(tmp...)
        tmpp *= (-1)^(sum(c .* b)) / binomial(n, sum(a))
        op += tmpp
    end
    op ./= 2^n
    return op
end 

function true_op(c)
    n = length(c)
    k = sum(c)
    ii = [1, 1]
    zz = [1, -1]
    tmp = n_choose_k(n, k)
    op = zeros(2^n)
    for b in tmp
        op += kron([b[i]==1 ? zz : ii for i in 1:n]...)
    end
    op ./= binomial(n, k)
    return op
end

true_op (generic function with 1 method)

In [3]:
c = [1, 1, 0, 0]
cal_op(c) - true_op(c)

16-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [4]:
function all_qs(n)
    if n==1
        return [0, 1, 2, 3]
    else
        tmp = all_qs(n-1)
        tmpp = Array{Int, 1}[]
        for a in tmp
            push!(tmpp, [0, a...])
            push!(tmpp, [1, a...])
            push!(tmpp, [2, a...])
            push!(tmpp, [3, a...])
        end
        return tmpp
    end
end

function charge_q_mask(n, q)
    tmp = n_choose_k(n, q)
    mask = zeros(Bool, 2^n)
    for a in tmp
        idx = sum([2^(i-1) * a[i] for i in 1:n]) + 1
        mask[idx] = true
    end
    return mask
end

function charge_q_indices(n, q)
    tmp = n_choose_k(n, q)
    mask = zeros(Bool, 2^n)
    indices = Int[]
    for a in tmp
        idx = sum([2^(i-1) * a[i] for i in 1:n]) + 1
        push!(indices, idx)
    end
    sort!(indices)
    return indices
end

"""
frome 1L 1R 2L 2R => 1L 2L 1R 2R
"""
function reorder_dims(op)
    n = Int(log2(length(op)) /2)
    good_order = [[2i-1 for i in 1:n]..., [2i for i in 1:n]...]
    tmp = reshape(op, [2 for _ in 1:2n]...)
    tmpp = permutedims(tmp, good_order)
    return reshape(tmpp, length(op))
end


function paired_state(n, perm)
    d = 2^n
    result = zeros(Int, d^4)
    for a1 in 1:d, a2 in 1:d
        if perm==false
            idx = (a1-1) * d^0 + (a1-1) * d^1 + (a2-1) * d^2 + (a2-1) * d^3
        else
            idx = (a1-1) * d^0 + (a2-1) * d^1 + (a2-1) * d^2 + (a1-1) * d^3
        end
        result[idx + 1] += 1
    end
    return result
end

# """
# Hilbert space:
# 1 \otimes 1bar \otimes 2 \otimes 2bar
# """
function charged_paired_state(n, perm, q1, q2)
    d = 2^n
    result = zeros(Int, d^4)
    indices1 = charge_q_indices(n, q1)
    indices2 = charge_q_indices(n, q2)
    for a1 in indices1, a2 in indices2
        if perm==false
            idx = (a1-1) * d^0 + (a1-1) * d^1 + (a2-1) * d^2 + (a2-1) * d^3
        else
            idx = (a1-1) * d^0 + (a2-1) * d^1 + (a2-1) * d^2 + (a1-1) * d^3
        end
        result[idx + 1] += 1
    end
    return result
end

function charged_2nd_frame_op(n)
    d = 2^n
    part1 = zeros(d^4 * d^4)
    part2 = zeros(d^4 * d^4)
    
    for q1 in 0:n, q2 in 0:n
        if q1==q2
            continue
        end
        tmp1 = charged_paired_state(n, false, q1, q2)
        tmp2 = charged_paired_state(n, true, q1, q2)
        part1 += (kron(tmp1, tmp1) + kron(tmp2, tmp2)) / (binomial(n, q1) * binomial(n, q2))
    end
    
    for q in 0:n
        dq = binomial(n, q)
        tmp1 = charged_paired_state(n, false, q, q)
        tmp2 = charged_paired_state(n, true, q, q)
        if dq==1
            part2 += kron(tmp1, tmp1)
        else
            part2 += (kron(tmp1, tmp1) + kron(tmp2, tmp2)) / (dq^2 - 1)
            part2 -= (kron(tmp1, tmp2) + kron(tmp2, tmp1)) / ((dq^2 - 1) * dq)
        end
    end
    result = part1 + part2
    result = reshape(result, d^4, d^4)
    return result
end

function component_square(a, W, b)
    n = Int(log2(size(W, 1))/4)
    a = reorder_dims(a)
    b = reorder_dims(b)
    return kron(a, a)' * W * kron(b, b)
end

function qs_to_pauli(qs)
    tmp = (II, X, Y, Z)
    tmpp = [tmp[i+1] for i in qs]
    return kron(tmpp...)
end

function qs_to_str(qs)
    tmp = ("I", "X", "Y", "Z")
    tmpp = [tmp[i+1] for i in qs]
    return *(tmpp...)
end

function component_square_analysis(W, b)
    n = Int(log2(size(W, 1))/4)
    eps = 1e-7
    qss = all_qs(n)
    for qs in qss
        str = qs_to_str(qs)
        pauli = qs_to_pauli(qs)
        tmp = component_square(pauli, W, b)
        if norm(tmp) > eps
            println("$(str) : $(tmp)")
        end
    end
    return nothing
end

function full_component_analysis(W)
    n = Int(log2(size(W, 1))/4)
    eps = 1e-7
    qss = all_qs(n)
    d = length(qss)
    all_paulis = [reorder_dims(qs_to_pauli(qs)) for qs in qss]
    result = zeros(Float64, d^2, d^2)
    for i1 in 1:d, i2 in 1:d
        p1, p2 = all_paulis[i1], all_paulis[i2]
        for j1 in 1:d, j2 in 1:d
            q1, q2 = all_paulis[j1], all_paulis[j2]
            tmp = kron(p1, p2)' * W * kron(q1, q2)
            if norm(tmp) > eps
                result[(i1-1) + (i2-1)*d + 1, (j1-1) + (j2-1)*d + 1] = tmp
            end
        end
    end
    return result
end

function analysis_report(n, result)
    eps = 1e-7
    qss = all_qs(n)
    d = length(qss)
    for i1 in 1:d, i2 in 1:i1
        a = (i1-1) + (i2-1)*d + 1
        str1 = qs_to_str(qss[i1]) * " " * qs_to_str(qss[i2])
        print("$(str1) : ")
        for j1 in 1:d, j2 in 1:d
            b = (j1-1) + (j2-1)*d + 1
            if norm(result[a,b]) > eps
                str2 = qs_to_str(qss[j1]) * "_" * qs_to_str(qss[j2])
                print("$(round(result[a,b], digits=2))$(str2) ")
            end
        end
        print("\n")
    end
    return nothing
end

analysis_report (generic function with 1 method)

In [5]:
W = charged_2nd_frame_op(3)
norm(W*W-W)^2

3.466673899897025e-32

In [72]:
W = charged_2nd_frame_op(3)
q = reorder_dims(kron(II, II, II))
qq = kron(q, q)
norm(W*qq - qq)^2

1.2040914012309e-30

In [13]:
q = reorder_dims(kron(II, II))
qq = kron(q, q)
qq' * charged_paired_state(2, true, 0, 0)

1

In [110]:
W = charged_2nd_frame_op(2)
component_square_analysis(W, kron(Z, II))

ZI : 5.333333333333334
XX : 1.3333333333333333
YX : 1.3333333333333333 + 0.0im
XY : 1.3333333333333333 + 0.0im
YY : 1.3333333333333333 + 0.0im
IZ : 5.333333333333334


In [75]:
W = charged_2nd_frame_op(3)
component_square_analysis(W, kron(Z, II, II))

ZII : 8.888888888888886
XXI : 1.3333333333333337
YXI : 1.3333333333333337 + 0.0im
XYI : 1.3333333333333337 + 0.0im
YYI : 1.3333333333333337 + 0.0im
IZI : 8.888888888888886
ZZI : 1.7777777777777783
XIX : 1.3333333333333337
YIX : 1.3333333333333337 + 0.0im
IXX : 1.3333333333333337
ZXX : 1.3333333333333337
IYX : 1.3333333333333337 + 0.0im
ZYX : 1.3333333333333337 + 0.0im
XZX : 1.3333333333333337
YZX : 1.3333333333333337 + 0.0im
XIY : 1.3333333333333337 + 0.0im
YIY : 1.3333333333333337 + 0.0im
IXY : 1.3333333333333337 + 0.0im
ZXY : 1.3333333333333337 + 0.0im
IYY : 1.3333333333333337 + 0.0im
ZYY : 1.3333333333333337 + 0.0im
XZY : 1.3333333333333337 + 0.0im
YZY : 1.3333333333333337 + 0.0im
IIZ : 8.888888888888886
ZIZ : 1.7777777777777783
XXZ : 1.3333333333333337
YXZ : 1.3333333333333337 + 0.0im
XYZ : 1.3333333333333337 + 0.0im
YYZ : 1.3333333333333337 + 0.0im
IZZ : 1.7777777777777775


In [96]:
n = 2
W = charged_2nd_frame_op(n)
result = full_component_analysis(W)

256×256 Array{Float64,2}:
 16.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  8.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0
  0.0  0.0  0.0  8.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0

In [109]:
analysis_report(2, result)

II II : 16.0II_II 
XI II : 
XI XI : 2.0XI_XI 2.0YI_YI 2.0IX_IX 2.0ZX_ZX 2.0IY_IY 2.0ZY_ZY 2.0XZ_XZ 2.0YZ_YZ 
YI II : 
YI XI : -2.0XI_YI 2.0YI_XI -2.0IX_IY -2.0ZX_ZY 2.0IY_IX 2.0ZY_ZX -2.0XZ_YZ 2.0YZ_XZ 
YI YI : 2.0XI_XI 2.0YI_YI 2.0IX_IX 2.0ZX_ZX 2.0IY_IY 2.0ZY_ZY 2.0XZ_XZ 2.0YZ_YZ 
ZI II : 8.0ZI_II 8.0IZ_II 
ZI XI : 
ZI YI : 
ZI ZI : 5.33ZI_ZI 2.67ZI_IZ 1.33XX_XX 1.33XX_YY 1.33YX_YX -1.33YX_XY -1.33XY_YX 1.33XY_XY 1.33YY_XX 1.33YY_YY 2.67IZ_ZI 5.33IZ_IZ 
IX II : 
IX XI : 
IX YI : 
IX ZI : 
IX IX : 2.0XI_XI 2.0YI_YI 2.0IX_IX 2.0ZX_ZX 2.0IY_IY 2.0ZY_ZY 2.0XZ_XZ 2.0YZ_YZ 
XX II : 
XX XI : 
XX YI : 
XX ZI : 
XX IX : 
XX XX : 1.33ZI_ZI -1.33ZI_IZ 3.33XX_XX -0.67XX_YY 3.33YX_YX 0.67YX_XY 0.67XY_YX 3.33XY_XY -0.67YY_XX 3.33YY_YY -1.33IZ_ZI 1.33IZ_IZ 
YX II : 
YX XI : 
YX YI : 
YX ZI : 
YX IX : 
YX XX : -2.0XX_YX -2.0XX_XY 2.0YX_XX -2.0YX_YY 2.0XY_XX -2.0XY_YY 2.0YY_YX 2.0YY_XY 
YX YX : 1.33ZI_ZI -1.33ZI_IZ 3.33XX_XX -0.67XX_YY 3.33YX_YX 0.67YX_XY 0.67XY_YX 3.33XY_XY -0.67YY_XX 3.33YY_YY -1.3

In [97]:
tmp1 = 0
n = 2
for q1 in 0:2, q2 in 0:2
    tmp1 = tmp1 .+ charged_paired_state(n, false, q1, q2)
end

In [99]:
@show tmp1
tmp2 = kron(reorder_dims(kron(II, II)),reorder_dims(kron(II, II)));
@show tmp2-tmp1;

tmp1 = [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
tmp2 - tmp1 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [103]:
d = 8
result1 = zeros(Int, 64)
for i in 1:d
    result1[(i-1)*d + i-1+1] =1
end
@show result1
result2 = reorder_dims(kron(II, II, II))
@show result2;

result1 = [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]
result2 = [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [39]:
q = reorder_dims(kron(II, II))
qq = kron(q, q)

W*qq - qq

256-element Array{Float64,1}:
 -1.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -1.0

In [53]:
all_qs(2)

8-element Array{Array{Int64,1},1}:
 [0, 1]
 [1, 1]
 [2, 1]
 [3, 1]
 [0, 0]
 [1, 0]
 [2, 0]
 [3, 0]

In [89]:
a = [0., 0.]

2-element Array{Float64,1}:
 0.0
 0.0

In [138]:
function haar_unitary(n, seed=nothing)
    eps = 1e-7
    while true
        if seed === nothing
            tmp = randn(n, n) + im .* randn(n, n)
        else
            rng = MersenneTwister(seed)
            tmp = randn(rng, n, n) + im .* randn(rng, n, n)
        end
        q, r = qr!(tmp)
        q = Array(q)
        ill_cond = false
        for i in 1:n
            tmp = abs(r[i ,i])
            if tmp < eps
                ill_cond = true
                break
            end
            q[:, i] *= r[i ,i] / tmp
        end
        ill_cond && continue
        return q
    end
end

function diag_swap()
    mat = zeros(ComplexF64, 4, 4)
    mat[1, 1] = exp(2pi*rand()*1im)
    mat[4, 4] = exp(2pi*rand()*1im)
    if rand() < 0.5
        mat[2, 2] = exp(2pi*rand()*1im)
        mat[3, 3] = exp(2pi*rand()*1im)
    else
        mat[2, 3] = exp(2pi*rand()*1im)
        mat[3, 2] = exp(2pi*rand()*1im)
    end
    return mat
end

function cc()
    mat = zeros(ComplexF64, 4, 4)
    mat[1, 1] = exp(2pi*rand()*1im)
    mat[4, 4] = exp(2pi*rand()*1im)
    mat[2:3, 2:3] = haar_unitary(2)
    return mat
end

cc (generic function with 1 method)

In [148]:
avg = 10000
begin
    a1 = kron(reshape(Y,2,2), [1 0; 0 1])
    a2 = kron(reshape(X,2,2), [1 0; 0 1])
    b1 = kron(reshape(X,2,2), [1 0; 0 1])
    b2 = kron(reshape(Y,2,2), [1 0; 0 1])
    tmp1 = kron(a1, a2)
    s = 0
    for _ in 1:avg
        U = diag_swap()
#         U = cc()
        tmp2 = kron(U * b1 * U', U * b2 * U')
        s += tr(tmp1 * tmp2)
    end
    s /= avg
    s
end

-1.9902940388468506 - 5.468471812528075e-19im

In [150]:
U = cc()
U * U'


4×4 Array{Complex{Float64},2}:
 1.0+0.0im           0.0+0.0im                   0.0+0.0im          0.0+0.0im
 0.0-0.0im           1.0+0.0im          -1.72066e-16-1.72393e-16im  0.0+0.0im
 0.0-0.0im  -1.72066e-16+1.72393e-16im           1.0+0.0im          0.0+0.0im
 0.0-0.0im           0.0-0.0im                   0.0-0.0im          1.0+0.0im

In [7]:
sum(eigvals(W))

29.999999999999996

In [9]:
eigen(W)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
4096-element Array{Float64,1}:
 -1.9615382426958483e-16
 -1.7718559110598632e-16
 -1.1082629677864934e-16
 -9.334604980189909e-17
 -8.546108206431453e-17
 -6.938893903907228e-17
 -6.582819108112845e-17
 -6.489665460404294e-17
 -6.381276237966998e-17
 -6.28897948319363e-17
 -6.105317554084703e-17
 -5.4263460305519584e-17
 -5.327841689042989e-17
  ⋮
  0.9999999999999999
  0.9999999999999999
  0.9999999999999999
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0000000000000002
vectors:
4096×4096 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  